# RAVIR-Challenge
RAVIR: A Dataset and Methodology for the Semantic Segmentation and Quantitative Analysis of Retinal Arteries and Veins in Infrared Reflectance Imaging Challenge.

## Introduction
This is an implementation to solve this challenge. This implementation is presented as a project for the subject DEEP LEARNING FOR MEDICAL IMAGE ANALYSIS (DLMIA) coursed in Universitat de Barcelona as a part of a Master Degree in Artificial Inteligence.

## Loading folders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## Imports

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import os
import random
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from keras import layers
from sklearn import metrics
from PIL import ImageOps
from scipy.spatial.distance import dice

## Prepare data

In [ ]:
input_dir_train = "RAVIR_Dataset\\train\\training_images"
target_dir_train = "RAVIR_Dataset\\train\\training_masks"
input_dir_train_da = "RAVIR_Dataset_DA\\training_images"
target_dir_train_da = "RAVIR_Dataset_DA\\training_masks"
input_dir_test = ""
target_dir_test = ""
img_size = (256, 256)
def get_paths(input_dir, target_dir):
    input_img_paths = sorted(
      [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".png")
      ]
    )
    target_img_paths = sorted(
      [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
      ]
    )
    return input_img_paths, target_img_paths

DATA_AUGMENTATION = True
VALIDATION = False
paths_train, paths_target = get_paths(input_dir_train, target_dir_train)
#paths_test , path_target_test = get_paths(input_dir_test, target_dir_test)
paths_train_da , paths_target_da = get_paths(input_dir_train_da, target_dir_train_da)
paths_train_val, paths_target_val = [],[]

if DATA_AUGMENTATION:
  paths_train += paths_train_da
  paths_target += paths_target_da
else:
  del paths_train_da, paths_target_da

if VALIDATION:
  split = int(len(paths_train)*0.8)
  paths_train_val = paths_train[split:]
  paths_train = paths_train[:split]
  paths_target_val = paths_target[split:]
  paths_target = paths_target[:split]

print("Total images:", len(paths_train))
print("Total target images:", len(paths_target))


## Data Generator

In [ ]:
class DataGenerator(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""
    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)
            x[j] = img
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            img = np.array(img)
            img[img==255] = 2
            img[img==128] = 1
            y[j] = np.expand_dims(img, 2)
        return x, y

## Model

Model used for this challenge

### Unet

In [ ]:
def unet(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model

## GENERATORS AND CONSTANTS TO TRAIN


In [ ]:
BATCHSIZE = 8
EPOCHS = 30
IMAGE_SIZE = (512,512)
lr = 0.001
optimizerA = keras.optimizers.Adam(learning_rate=lr)
optimizerRMSPROP = keras.optimizers.RMSprop(learning_rate=lr)


train_generator = DataGenerator(BATCHSIZE, IMAGE_SIZE, paths_train, paths_target)


## Training

In [ ]:
model = unet(IMAGE_SIZE, num_classes=3)

In [ ]:
model.compile(optimizer=optimizerA, loss="sparse_categorical_crossentropy", metrics=["accuracy" ])
callbacks = [
    keras.callbacks.ModelCheckpoint("ravir_challenge.h5", save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='loss', min_delta=1e-4, patience=5, verbose=1, mode='auto', restore_best_weights=True)
]
if VALIDATION:
    val_generator = DataGenerator(BATCHSIZE, IMAGE_SIZE, paths_train_val, paths_target_val)
    mdl_hist = model.fit(train_generator, epochs=EPOCHS, validation_data=val_generator, callbacks=callbacks)
else:
    mdl_hist = model.fit(train_generator, epochs=EPOCHS, callbacks=callbacks)

## Plotting results of training

In [ ]:
plt.plot(mdl_hist.history['loss'], label='train loss')
if VALIDATION:
    plt.plot(mdl_hist.history['val_loss'], label='val loss')
plt.legend()
plt.savefig("unet_data_epochs_" + str(EPOCHS) + "_batchSize_"+ str(BATCHSIZE) + '_LossVal_loss')
plt.show()
# plot the accuracy
plt.plot(mdl_hist.history['accuracy'], label='train acc')
if VALIDATION:
    plt.plot(mdl_hist.history['val_accuracy'], label='val acc')
plt.legend()
plt.savefig("unet_data_epochs_" +str(EPOCHS)+"_batchSize_"+ str(BATCHSIZE)+'_AccVal_acc')
#save model to disk
plt.show()

## Testing

In [ ]:
model.compile(optimizer=optimizerA, loss="sparse_categorical_crossentropy", metrics=["accuracy" ])
model.load_weights('ravir_challenge_bo_da.h5')

In [ ]:

results = model.evaluate(train_generator, batch_size=8)
print("test loss, test acc:", results)
val_preds = model.predict(train_generator)

def mask_to_image(preds, i):
    """Quick utility to display a model's prediction."""
    mask = np.argmax(preds[i], axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    img = keras.preprocessing.image.array_to_img(mask)
    return img


import numpy.ma as ma

# Display results for validation image #10
inds = [0,2,4,8]

# Create figure and plot
fig = plt.figure(figsize=(14, 10))

nrows = len(inds)
ncols = 3

for row, i in enumerate(inds):
    base_ind = row*ncols

    # Add the original image subplot at the 1st position
    fig.add_subplot(nrows, ncols, base_ind+1)
    image = plt.imread(paths_train[i])
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.title("Original Image")

    # Add the original target subplot at the 2nd position
    fig.add_subplot(nrows, ncols, base_ind+2)
    target = plt.imread(paths_target[i])
    plt.imshow(target, cmap="gray")
    plt.axis('off')
    plt.title("Target")

    # Add the predicted target subplot at the 3rd position
    fig.add_subplot(nrows, ncols, base_ind+3)
    pred = mask_to_image(val_preds, i)
    plt.imshow(pred, cmap="gray")
    plt.axis('off')
    plt.title("Prediction")

plt.show()

## Computing metrics Sensitivity, Specificity, Accuracy, Jaccard and Dice

In [ ]:
from sklearn.metrics import accuracy_score, recall_score

def get_values(y_true, y_pred):
    # True Positives
    tp = np.sum(np.logical_and(y_pred == 1, y_true == 1))
    # True Negatives
    tn = np.sum(np.logical_and(y_pred == 0, y_true == 0))
    # False Positives
    fp = np.sum(np.logical_and(y_pred == 1, y_true == 0))
    # False Negatives
    fn = np.sum(np.logical_and(y_pred == 0, y_true == 1))
    return tn, fp, fn, tp

def sensitivity(y_true, y_pred):
    #return metrics.recall_score(y_true, y_pred, zero_division=0,average='micro')
    tn, fp, fn, tp = get_values(y_true, y_pred) 
    res = (tp)/(tp+fn)
    return res

def specificity(y_true , y_pred, epsilon=1e-20):
    tn, fp, _, _ = get_values(y_true, y_pred)
    res = (tn)/(tn+fp+epsilon)
    return res

def accuracy(y_true, y_pred, epsilon=1e-20):
    tn, fp, fn, tp = get_values(y_true, y_pred)
    res = (tp+tn)/(tp+fp+tn+fn+epsilon)
    return res

def jaccard(y_true, y_pred):
    tn, fp, fn, tp = get_values(y_true, y_pred)

    return (tp)/(tp+fp+fn)

def dice_coef(y_true, y_pred, epsilon=1e-20):
    _ , fp, fn, tp = get_values(y_true, y_pred)
    res = (2*tp)/(2*tp+fp+fn+epsilon)
    return res
if VALIDATION:
  test_generator = val_generator
else:
  test_generator = DataGenerator(1, IMAGE_SIZE, paths_train, paths_target)

values = []
results = model.predict(test_generator)
print("############################################")
sensitivity_val = 0
accuracy_val = 0
specificity_val = 0
jaccard_val = 0
dice_val = 0


for i in range(len(results)):
    img = results[i]
    img_true = np.array(load_img(paths_target_val[i],target_size=(512,512),color_mode="grayscale"))
    img_true = img_true/255
    mask = np.argmax(img, axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    img = keras.preprocessing.image.array_to_img(mask)
    img = np.array(img)
    img = np.around(img)
    img = img/255
    #print(img_true)
    sensitivity_val += sensitivity(img_true, img)
    specificity_val += specificity(img_true, img)
    accuracy_val += accuracy(img_true, img)
    jaccard_val += jaccard(img_true,img)
    dice_val += dice_coef(img_true, img)
values.append([sensitivity_val/len(results), specificity_val/len(results), accuracy_val/len(results), jaccard_val/len(results), dice_val/len(results)])
print("Sensitivity:", sensitivity_val/len(results))
print("Specificity:", specificity_val/len(results))
print("Accuracy:", accuracy_val/len(results))
print("jaccard:", jaccard_val/len(results))
print("dice_val:", dice_val/len(results))